In [ ]:
# 참고 코드
'''
import os
import telebot
import yfinance as yf

API_KEY = os.getenv('API_KEY')
bot = telebot.TeleBot(API_KEY)

@bot.message_handler(commands=['Greet'])
def greet(message):
  bot.reply_to(message, "Hey! Hows it going?")

@bot.message_handler(commands=['hello'])
def hello(message):
  bot.send_message(message.chat.id, "Hello!")

@bot.message_handler(commands=['wsb'])
def get_stocks(message):
  response = ""
  stocks = ['gme', 'amc', 'nok']
  stock_data = []
  for stock in stocks:
    data = yf.download(tickers=stock, period='2d', interval='1d')
    data = data.reset_index()
    response += f"-----{stock}-----\n"
    stock_data.append([stock])
    columns = ['stock']
    for index, row in data.iterrows():
      stock_position = len(stock_data) - 1
      price = round(row['Close'], 2)
      format_date = row['Date'].strftime('%m/%d')
      response += f"{format_date}: {price}\n"
      stock_data[stock_position].append(price)
      columns.append(format_date)
    print()

  response = f"{columns[0] : <10}{columns[1] : ^10}{columns[2] : >10}\n"
  for row in stock_data:
    response += f"{row[0] : <10}{row[1] : ^10}{row[2] : >10}\n"
  response += "\nStock Data"
  print(response)
  bot.send_message(message.chat.id, response)

def stock_request(message):
  request = message.text.split()
  if len(request) < 2 or request[0].lower() not in "price":
    return False
  else:
    return True

@bot.message_handler(func=stock_request)
def send_price(message):
  request = message.text.split()[1]
  data = yf.download(tickers=request, period='5m', interval='1m')
  if data.size > 0:
    data = data.reset_index()
    data["format_date"] = data['Datetime'].dt.strftime('%m/%d %I:%M %p')
    data.set_index('format_date', inplace=True)
    print(data.to_string())
    bot.send_message(message.chat.id, data['Close'].to_string(header=False))
  else:
    bot.send_message(message.chat.id, "No data!?")

bot.polling()
'''

In [1]:
# 라이브러리 및 api 설정
import telebot
import requests
from bs4 import BeautifulSoup

API_KEY = '1644820427:AAEllDn4xGfRAFHGpb9lFxx5QsK-__AG7L8'
bot = telebot.TeleBot(API_KEY)
chid_id = '428116987'

@bot.message_handler(commands=['Greet'])
def greet(message):
  bot.reply_to(message, "Hey! Hows it going?")

# 주간학습 크롤링 봇
@bot.message_handler(commands=['jugan12'])
def check_jugan12(message):
    p_url = "http://www.asbugok.es.kr/board.list?mcode=272635&cate=272635"
    result = ""
    res = requests.get(p_url, verify=False) #ssl 인증서 관련 
    res.raise_for_status()  # 정보가 없으면 프로그램 종료
    soup = BeautifulSoup(res.text, "lxml")
    # soup = BeautifulSoup(html, 'html.parser')
    class_name = soup.find("div", {"class" : "titleTop"})
    table = soup.find("table", {"class" : "boardList"})
    titles = table.find_all('td', {'class' : "title"})
    _class = class_name.find('img')['alt']  # 반이름 출력
    result += f'{_class}\n'
    try:
        title = titles[0].find('a').text    # 가장 최신의 글 저장
        result += f'{title}\n'
        # time.sleep(2)
        bot.send_message(message.chat.id, result)
    except:
        result += '에러발생'
        # time.sleep(2)
        bot.send_message(message.chat.id, 'hi')

# 뉴스 크롤링 봇
@bot.message_handler(commands=['news'])
def today_news(message):
    # 크롤링 정보 받아오기
    def create_soup(p_url):
        res = requests.get(p_url, verify=False) #ssl 인증서 관련 
        res.raise_for_status()  # 정보가 없으면 프로그램 종료
        soup = BeautifulSoup(res.text, "lxml")
        return soup

    try:
        result = ""
        result += f"===== 이시간 주요 뉴스 =====\n"
        URL = "https://news.daum.net"
        soup = create_soup(URL)  # 뷰티플 숩 객체 만들기

        box_headline = soup.find("div", {"class": "box_headline"})
        lis = box_headline.select('ul > li')
        i = 1

        for li in lis:
            content = li.find_all('strong')
            txt = content[0].text.strip().replace('\n', " / ")
            link = li.find('a').get('href')
            result += f'{i}. {txt} / {link}\n'
            i += 1
        bot.send_message(message.chat.id, result)

    except Exception as e:
           bot.send_message(message.chat.id, '뉴스 정보 수집 중 오류가 발생했습니다.')

bot.polling()